In [1]:
from util import kafkaProducer

In [2]:
csv_path="data/camera_event_a.csv"
#ip address is the zookeeper & port to the kafka port
kafka_server="172.17.0.1:9092"
producer_id="producer_a"
topic="camera_event_a"
batch_interval=5

producer = kafkaProducer(csv_path, kafka_server, producer_id, topic, batch_interval)
producer.produce_batches()

[INFO] Successfully loaded data from data/camera_event_a.csv
print 172.17.0.1:9092
172.17.0.1:9092
[INFO] Publishing batch #1 (20 records)...
[INFO] Batch #1 sent. Sleeping 5s...
[DATA] Batch #1:
                                event_id  batch_id car_plate  camera_id  \
0   d40c586c-5be6-4743-a1e3-2269d9edaa72         1     KRN 7          1   
1   85c08e3c-a0b5-45d8-a70c-df8f9a6d5829         1     ICE 8          1   
2   f5834b79-771b-4931-8da2-a5ad7f4ccd02         1   QE 1820          1   
3   d0e547bb-c4a7-4750-b7b4-8076e9b47f4f         1   CJW 924          1   
4   f3162606-1b2e-407f-951d-61d14c0a7b09         1   CJP 278          1   
5   c69852f7-cd9a-4892-b225-8f8a36ec017b         1    ZPG 90          1   
6   5113d990-4da6-41d2-b292-2516e7d8dc07         1    KKE 15          1   
7   67ce0672-95b5-43bd-9940-2e16db5f747a         1     WJX 2          1   
8   7c5c112b-7521-4bbc-90f8-3c3a85cc3fe1         1     FO 32          1   
9   213f0627-2be4-409c-972b-669627ca0461         1    

KeyboardInterrupt: 